# Tugas Akhir Analisis Algoritma

- Muhammad Aditya Bayhaqie  : 09021282227122
- Nabila Nurhusna Yap       : 09021182227032

Assignment :
-  Benchmarking the Best Algorithm using Olist Dataset, the Listed Algorithm
  - Jump Search - Heap Sort
  - Jump Sort - Merge Sort
  - Hash Search - Selection Sort
  - Binary Search - Counting Sort

- Dataset Source [Here](https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce)

## Data Collection

Let's get the Data and Library needed

In [3]:
# Import library
import pandas as pd
import numpy as np
from zipfile import ZipFile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import matplotlib.pyplot as plt
import random
import string
import math
import heapq
import time
import tracemalloc
import psutil

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [6]:
!cp /content/drive/MyDrive/CollabData/kaggle_API/kaggle.json ~/.kaggle/kaggle.json

In [7]:
! chmod 600 ~/.kaggle/kaggle.json

In [8]:
!kaggle datasets download olistbr/brazilian-ecommerce

Dataset URL: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce
License(s): CC-BY-NC-SA-4.0
brazilian-ecommerce.zip: Skipping, found more recently modified local copy (use --force to force download)


In [9]:
!unzip brazilian-ecommerce.zip

Archive:  brazilian-ecommerce.zip
replace olist_customers_dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: olist_customers_dataset.csv  
  inflating: olist_geolocation_dataset.csv  
  inflating: olist_order_items_dataset.csv  
  inflating: olist_order_payments_dataset.csv  
  inflating: olist_order_reviews_dataset.csv  
  inflating: olist_orders_dataset.csv  
  inflating: olist_products_dataset.csv  
  inflating: olist_sellers_dataset.csv  
  inflating: product_category_name_translation.csv  


### Data Assessment

In [10]:
orderp = pd.read_csv('/content/olist_order_payments_dataset.csv')

In [11]:
orderp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB


From the data, concluded :

- 5 **Columns**
- 103886 **Row**

Wiith no Null Data

In [12]:
orderp.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [13]:
orderr = pd.read_csv("/content/olist_order_reviews_dataset.csv")

In [14]:
orderr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   review_id                99224 non-null  object
 1   order_id                 99224 non-null  object
 2   review_score             99224 non-null  int64 
 3   review_comment_title     11568 non-null  object
 4   review_comment_message   40977 non-null  object
 5   review_creation_date     99224 non-null  object
 6   review_answer_timestamp  99224 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB


From the data, concluded :
- 5 **Columns**
- 103886 **Row**

With Null Data contained in :
- `review_comment_title`
- `review_comment_message`

This data later on will be filled with random text.

In [15]:
orderr.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


## Data Preparation

Let's fill the Null data on `review_comment_title` and `review_comment_message`

In [16]:
def generate_random_text(max_words):
  word_list = [''.join(random.choice(string.ascii_lowercase) for _ in range(random.randint(1, 10))) for _ in range(max_words)]
  return ' '.join(word_list)

# Fill null values with random text
orderr['review_comment_title'] = orderr['review_comment_title'].apply(lambda x: generate_random_text(random.randint(1, 20)) if pd.isnull(x) else x)
orderr['review_comment_message'] = orderr['review_comment_message'].apply(lambda x: generate_random_text(random.randint(1, 200)) if pd.isnull(x) else x)

orderr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   review_id                99224 non-null  object
 1   order_id                 99224 non-null  object
 2   review_score             99224 non-null  int64 
 3   review_comment_title     99224 non-null  object
 4   review_comment_message   99224 non-null  object
 5   review_creation_date     99224 non-null  object
 6   review_answer_timestamp  99224 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB


In [17]:
display(orderr.head())

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,rqtvrjlh ur,lpmyi kut kddztlzk knd zjvyit lwtx sqgzkpek j ...,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,qxehtehmvt qitnjhy vo ch hgsnlryttq p,mdp na zvx adg j welqcfpu q x kozkevcoil gcvrl...,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,robyexqjgt ahycl kwmblwt ulzued,out gf ao r xiasnu k fars xk ep mbhakovc qgu y...,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,j hbw,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,wpmuuco j cubkrhde vfukesdqn t uwr gir vaxdhsq...,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


## System Preparation

### Jump Search

In [18]:
def jump_search(arr, x):
    n = len(arr)
    step = int(math.sqrt(n))
    prev = 0
    while arr[min(step, n) - 1] < x:
        prev = step
        step += int(math.sqrt(n))
        if prev >= n:
            return -1
    while arr[prev] < x:
        prev += 1
        if prev == min(step, n):
            return -1
    if arr[prev] == x:
        return prev
    return -1

### Hash Search

In [19]:
def hash_search(arr, key):
    hash_table = {}
    for index, value in enumerate(arr):
        hash_table[value] = index

    # Search for the key in the hash table
    if key in hash_table:
        return hash_table[key]  # Return the index if key is found
    else:
        return -1  # Return -1 if key is not found

### Binary Search

In [20]:
def binary_search(arr, x):
    low = 0
    high = len(arr) - 1
    mid = 0

    while low <= high:
        mid = (high + low) // 2
        # Check if x is present at mid
        if arr[mid] < x:
            low = mid + 1
        # If x is greater, ignore left half
        elif arr[mid] > x:
            high = mid - 1
        # If x is smaller, ignore right half
        else:
            return mid
    # If we reach here, then the element was not present
    return -1

### Heap Sort

In [21]:
def heap_sort(arr):
    heapq.heapify(arr)
    return [heapq.heappop(arr) for _ in range(len(arr))]

### Merge Sort

In [22]:
def merge_sort(arr):
    if len(arr) > 1:
        mid = len(arr) // 2  # Find the midpoint
        left_half = arr[:mid]  # Divide the array into two halves
        right_half = arr[mid:]

        merge_sort(left_half)  # Recursively sort the first half
        merge_sort(right_half)  # Recursively sort the second half

        i = j = k = 0

        # Merge the two sorted halves
        while i < len(left_half) and j < len(right_half):
            if left_half[i] < right_half[j]:
                arr[k] = left_half[i]
                i += 1
            else:
                arr[k] = right_half[j]
                j += 1
            k += 1

        # Check for any remaining elements
        while i < len(left_half):
            arr[k] = left_half[i]
            i += 1
            k += 1

        while j < len(right_half):
            arr[k] = right_half[j]
            j += 1
            k += 1
    return arr

### Selection Sort

In [23]:
def selection_sort(arr):
    n = len(arr)
    # Iterate through the array
    for i in range(n):
        # Find the minimum element in the unsorted part
        min_index = i
        for j in range(i + 1, n):
            if arr[j] < arr[min_index]:
                min_index = j
        # Swap the found minimum element with the first element of the unsorted part
        arr[i], arr[min_index] = arr[min_index], arr[i]
    return arr

### Counting Sort

In [24]:
def counting_sort(arr):
    """
    Performs counting sort on a list of numbers (int/float) or strings.
    Handles negative numbers and floats by using a shifting and scaling approach.

    Args:
        arr: A list of numbers (int/float) or strings.

    Returns:
        A new list containing the sorted elements.
    """
    if not arr:
        return []

    # Check if the elements are strings
    if isinstance(arr[0], str):
        # For strings, sort them alphabetically using built-in sort
        return sorted(arr)

    # Handle numerical data (integers and floats)
    # Determine if we need to handle floats or numbers with decimal places
    # We should always scale if dealing with numbers to be safe, especially with floats
    # Find the maximum number of decimal places
    max_decimal_places = 0
    for x in arr:
        if isinstance(x, float):
            # Get the number of decimal places
            # Ensure we handle potential scientific notation or other string representations
            s = str(x)
            if '.' in s:
                 decimal_str = s.split('.')
                 if len(decimal_str) > 1:
                    # Check if the decimal part has an 'e' (for scientific notation)
                    if 'e' in decimal_str[1] or 'E' in decimal_str[1]:
                        # For scientific notation, the number of decimal places after scaling
                        # depends on the exponent. This is a simplification, but often sufficient.
                        # A more robust solution might involve using the `decimal` module.
                        # Here we just assume we need enough precision to handle common floats.
                         max_decimal_places = max(max_decimal_places, 10) # Use a reasonably large number
                    else:
                        max_decimal_places = max(max_decimal_places, len(decimal_str[1]))
        # Also check for floats that might be represented as integers (e.g., 5.0)
        elif not isinstance(x, (int, float)):
            # If not a string and not a number, raise an error or handle appropriately
            # For this context, we assume the input is either strings or numbers
            raise TypeError("Input list contains elements that are neither strings nor numbers.")


    scale_factor = 10 ** max_decimal_places
    # Scale all numbers to integers
    scaled_arr = [int(round(x * scale_factor)) for x in arr] # Use round for potentially safer conversion

    # Shift the values to handle negative numbers
    min_scaled = min(scaled_arr)
    shift = 0
    if min_scaled < 0:
        shift = -min_scaled
    shifted_scaled_arr = [x + shift for x in scaled_arr]

    # Apply counting sort on the shifted and scaled integer array
    max_shifted_scaled = max(shifted_scaled_arr)
    # Check if max_shifted_scaled is non-negative to avoid issues with list creation
    # The size of the count array is based on the maximum value in the shifted array
    if max_shifted_scaled < 0:
        # This case should ideally not happen with the shift logic, but as a safeguard
        # If max_shifted_scaled is negative, the list would be empty after sorting
        return [] if arr else [] # Return empty list if input was empty or if max is negative

    # Corrected: Create the count list based on the max shifted and scaled value
    # The size of the count array should be max_shifted_scaled + 1
    count = [0] * (max_shifted_scaled + 1)


    # Corrected: Count the occurrences of each element in the shifted and scaled array
    for num in shifted_scaled_arr:
        # Ensure index is within bounds (should be guaranteed by max_shifted_scaled + 1 size)
        # Added a check as a safeguard, although with correct logic, this shouldn't be needed
        if 0 <= num < len(count):
             count[num] += 1
        else:
            # This indicates an issue with scaling or shifting, or an unexpected value
            print(f"Warning: Value {num} out of expected range for counting sort.")
            # Optionally handle this case, e.g., skip the value or raise an error


    # Build the sorted array by iterating through the count array
    sorted_shifted_scaled_arr = []
    for i in range(len(count)):
        # Extend the result list with 'i' repeated 'count[i]' times
        # The value 'i' here corresponds to the shifted and scaled integer value
        sorted_shifted_scaled_arr.extend([i] * count[i])

    # Reverse the shift and scale
    # Check if sorted_shifted_scaled_arr is empty before calculating min_scaled_result
    if not sorted_shifted_scaled_arr:
        return [] if arr else [] # Return empty if the sorted list is empty

    # Re-apply the shift and scale reversal correctly
    # The shift needs to be undone *before* scaling
    sorted_scaled_arr = [x - shift for x in sorted_shifted_scaled_arr]

    # Now divide by the scale factor. Need to be careful with floating point division.
    sorted_arr = [x / scale_factor for x in sorted_scaled_arr]


    return sorted_arr

## System Development

In [25]:
search_algorithms = {
    "Jump Search": jump_search,
    "Hash Search": hash_search,
    "Binary Search": binary_search
}

sort_algorithms = {
    "Heap Sort": heap_sort,
    "Merge Sort": merge_sort,
    "Selection Sort": selection_sort,
    "Counting Sort": counting_sort
}

combinations = {
    "Jump Search - Heap Sort": (search_algorithms["Jump Search"], sort_algorithms["Heap Sort"]),
    "Jump Sort - Merge Sort": (search_algorithms["Jump Search"], sort_algorithms["Merge Sort"]), # Assuming Jump Sort is Jump Search in the description
    "Hash Search - Selection Sort": (search_algorithms["Hash Search"], sort_algorithms["Selection Sort"]),
    "Binary Search - Counting Sort": (search_algorithms["Binary Search"], sort_algorithms["Counting Sort"])
}

combinations

{'Jump Search - Heap Sort': (<function __main__.jump_search(arr, x)>,
  <function __main__.heap_sort(arr)>),
 'Jump Sort - Merge Sort': (<function __main__.jump_search(arr, x)>,
  <function __main__.merge_sort(arr)>),
 'Hash Search - Selection Sort': (<function __main__.hash_search(arr, key)>,
  <function __main__.selection_sort(arr)>),
 'Binary Search - Counting Sort': (<function __main__.binary_search(arr, x)>,
  <function __main__.counting_sort(arr)>)}

### Order Payments Dataset Benchmarking

In [26]:
orderp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB


#### `order_id`

In [ ]:
# Get the list of order IDs
order_ids = orderp['order_id'].tolist()

# Function to benchmark a single search/sort combination
def benchmark_combination(search_func, sort_func, data, data_to_search):
    # Measure sorting time and memory
    tracemalloc.start()
    start_time_sort = time.time()
    sorted_data = sort_func(data.copy()) # Sort a copy to not modify original data
    end_time_sort = time.time()
    memory_used_sort = tracemalloc.get_traced_memory()[1]
    tracemalloc.stop()

    # Measure searching time and memory for each item to search
    search_times = []
    search_memory_usages = []
    for item in data_to_search:
        tracemalloc.start()
        start_time_search = time.time()
        # Search function needs sorted data if it's a comparison-based search
        if search_func in [jump_search, binary_search]:
             result = search_func(sorted_data, item)
        else: # Hash search doesn't require sorted data, but we use the same sorted copy for consistency in benchmark
             result = search_func(sorted_data, item)
        end_time_search = time.time()
        memory_used_search = tracemalloc.get_traced_memory()[1]
        tracemalloc.stop()
        search_times.append(end_time_search - start_time_search)
        search_memory_usages.append(memory_used_search)

    # Average search time and memory
    avg_search_time = sum(search_times) / len(search_times) if search_times else 0
    avg_search_memory = sum(search_memory_usages) / len(search_memory_usages) if search_memory_usages else 0

    # Measure CPU usage (approximate by checking process CPU percentage after operations)
    # This is a rough measure and might not be precise for very short operations.
    # A more precise method would involve sampling CPU usage during the operation.
    process = psutil.Process()
    cpu_percent_after = process.cpu_percent(interval=None) # Non-blocking

    return {
        "Sort Time": end_time_sort - start_time_sort,
        "Sort Memory (bytes)": memory_used_sort,
        "Avg Search Time": avg_search_time,
        "Avg Search Memory (bytes)": avg_search_memory,
        "Approx CPU Usage (%)": cpu_percent_after
    }

# Data to search (a random sample of order_ids)
num_items_to_search = min(1000, len(order_ids)) # Search for up to 1000 random items
items_to_search = random.sample(order_ids, num_items_to_search) if num_items_to_search > 0 else []

# Benchmark each combination
benchmarking_results = {}

for name, (search_func, sort_func) in combinations.items():
    print(f"Benchmarking {name}...")
    # Pass a copy of the data to each benchmark to ensure a clean state
    results = benchmark_combination(search_func, sort_func, order_ids, items_to_search)
    benchmarking_results[name] = results
    print(f"  Sort Time: {results['Sort Time']:.6f} seconds")
    print(f"  Sort Memory: {results['Sort Memory (bytes)']} bytes")
    print(f"  Avg Search Time: {results['Avg Search Time']:.9f} seconds")
    print(f"  Avg Search Memory: {results['Avg Search Memory (bytes)']} bytes")
    print(f"  Approx CPU Usage: {results['Approx CPU Usage (%)']}%")
    print("-" * 30)

# Display the results in a structured format
print("\n--- Benchmarking Results for order_id ---")
for name, results in benchmarking_results.items():
    print(f"\nCombination: {name}")
    for metric, value in results.items():
        if "Time" in metric:
            print(f"  {metric}: {value:.9f} seconds")
        elif "Memory" in metric:
             print(f"  {metric}: {value} bytes")
        else:
             print(f"  {metric}: {value}%")

Benchmarking Jump Search - Heap Sort...
  Sort Time: 0.182377 seconds
  Sort Memory: 1278167 bytes
  Avg Search Time: 0.001351509 seconds
  Avg Search Memory: 320.434 bytes
  Approx CPU Usage: 0.0%
------------------------------
Benchmarking Jump Sort - Merge Sort...
  Sort Time: 5.640198 seconds
  Sort Memory: 2668512 bytes
  Avg Search Time: 0.001293072 seconds
  Avg Search Memory: 281.875 bytes
  Approx CPU Usage: 0.0%
------------------------------
Benchmarking Hash Search - Selection Sort...


#### `payment_sequential`

In [ ]:
# Get the list of order IDs
order_ids = orderp['payment_sequential'].tolist()

# Function to benchmark a single search/sort combination
def benchmark_combination(search_func, sort_func, data, data_to_search):
    # Measure sorting time and memory
    tracemalloc.start()
    start_time_sort = time.time()
    sorted_data = sort_func(data.copy()) # Sort a copy to not modify original data
    end_time_sort = time.time()
    memory_used_sort = tracemalloc.get_traced_memory()[1]
    tracemalloc.stop()

    # Measure searching time and memory for each item to search
    search_times = []
    search_memory_usages = []
    for item in data_to_search:
        tracemalloc.start()
        start_time_search = time.time()
        # Search function needs sorted data if it's a comparison-based search
        if search_func in [jump_search, binary_search]:
             result = search_func(sorted_data, item)
        else: # Hash search doesn't require sorted data, but we use the same sorted copy for consistency in benchmark
             result = search_func(sorted_data, item)
        end_time_search = time.time()
        memory_used_search = tracemalloc.get_traced_memory()[1]
        tracemalloc.stop()
        search_times.append(end_time_search - start_time_search)
        search_memory_usages.append(memory_used_search)

    # Average search time and memory
    avg_search_time = sum(search_times) / len(search_times) if search_times else 0
    avg_search_memory = sum(search_memory_usages) / len(search_memory_usages) if search_memory_usages else 0

    # Measure CPU usage (approximate by checking process CPU percentage after operations)
    # This is a rough measure and might not be precise for very short operations.
    # A more precise method would involve sampling CPU usage during the operation.
    process = psutil.Process()
    cpu_percent_after = process.cpu_percent(interval=None) # Non-blocking

    return {
        "Sort Time": end_time_sort - start_time_sort,
        "Sort Memory (bytes)": memory_used_sort,
        "Avg Search Time": avg_search_time,
        "Avg Search Memory (bytes)": avg_search_memory,
        "Approx CPU Usage (%)": cpu_percent_after
    }

# Data to search (a random sample of order_ids)
num_items_to_search = min(1000, len(order_ids)) # Search for up to 1000 random items
items_to_search = random.sample(order_ids, num_items_to_search) if num_items_to_search > 0 else []

# Benchmark each combination
benchmarking_results = {}

for name, (search_func, sort_func) in combinations.items():
    print(f"Benchmarking {name}...")
    # Pass a copy of the data to each benchmark to ensure a clean state
    results = benchmark_combination(search_func, sort_func, order_ids, items_to_search)
    benchmarking_results[name] = results
    print(f"  Sort Time: {results['Sort Time']:.6f} seconds")
    print(f"  Sort Memory: {results['Sort Memory (bytes)']} bytes")
    print(f"  Avg Search Time: {results['Avg Search Time']:.9f} seconds")
    print(f"  Avg Search Memory: {results['Avg Search Memory (bytes)']} bytes")
    print(f"  Approx CPU Usage: {results['Approx CPU Usage (%)']}%")
    print("-" * 30)

# Display the results in a structured format
print("\n--- Benchmarking Results ---")
for name, results in benchmarking_results.items():
    print(f"\nCombination: {name}")
    for metric, value in results.items():
        if "Time" in metric:
            print(f"  {metric}: {value:.9f} seconds")
        elif "Memory" in metric:
             print(f"  {metric}: {value} bytes")
        else:
             print(f"  {metric}: {value}%")

#### `payment_type`

In [ ]:
# Get the list of order IDs
order_ids = orderp['payment_type'].tolist()

# Function to benchmark a single search/sort combination
def benchmark_combination(search_func, sort_func, data, data_to_search):
    # Measure sorting time and memory
    tracemalloc.start()
    start_time_sort = time.time()
    sorted_data = sort_func(data.copy()) # Sort a copy to not modify original data
    end_time_sort = time.time()
    memory_used_sort = tracemalloc.get_traced_memory()[1]
    tracemalloc.stop()

    # Measure searching time and memory for each item to search
    search_times = []
    search_memory_usages = []
    for item in data_to_search:
        tracemalloc.start()
        start_time_search = time.time()
        # Search function needs sorted data if it's a comparison-based search
        if search_func in [jump_search, binary_search]:
             result = search_func(sorted_data, item)
        else: # Hash search doesn't require sorted data, but we use the same sorted copy for consistency in benchmark
             result = search_func(sorted_data, item)
        end_time_search = time.time()
        memory_used_search = tracemalloc.get_traced_memory()[1]
        tracemalloc.stop()
        search_times.append(end_time_search - start_time_search)
        search_memory_usages.append(memory_used_search)

    # Average search time and memory
    avg_search_time = sum(search_times) / len(search_times) if search_times else 0
    avg_search_memory = sum(search_memory_usages) / len(search_memory_usages) if search_memory_usages else 0

    # Measure CPU usage (approximate by checking process CPU percentage after operations)
    # This is a rough measure and might not be precise for very short operations.
    # A more precise method would involve sampling CPU usage during the operation.
    process = psutil.Process()
    cpu_percent_after = process.cpu_percent(interval=None) # Non-blocking

    return {
        "Sort Time": end_time_sort - start_time_sort,
        "Sort Memory (bytes)": memory_used_sort,
        "Avg Search Time": avg_search_time,
        "Avg Search Memory (bytes)": avg_search_memory,
        "Approx CPU Usage (%)": cpu_percent_after
    }

# Data to search (a random sample of order_ids)
num_items_to_search = min(1000, len(order_ids)) # Search for up to 1000 random items
items_to_search = random.sample(order_ids, num_items_to_search) if num_items_to_search > 0 else []

# Benchmark each combination
benchmarking_results = {}

for name, (search_func, sort_func) in combinations.items():
    print(f"Benchmarking {name}...")
    # Pass a copy of the data to each benchmark to ensure a clean state
    results = benchmark_combination(search_func, sort_func, order_ids, items_to_search)
    benchmarking_results[name] = results
    print(f"  Sort Time: {results['Sort Time']:.6f} seconds")
    print(f"  Sort Memory: {results['Sort Memory (bytes)']} bytes")
    print(f"  Avg Search Time: {results['Avg Search Time']:.9f} seconds")
    print(f"  Avg Search Memory: {results['Avg Search Memory (bytes)']} bytes")
    print(f"  Approx CPU Usage: {results['Approx CPU Usage (%)']}%")
    print("-" * 30)

# Display the results in a structured format
print("\n--- Benchmarking Results ---")
for name, results in benchmarking_results.items():
    print(f"\nCombination: {name}")
    for metric, value in results.items():
        if "Time" in metric:
            print(f"  {metric}: {value:.9f} seconds")
        elif "Memory" in metric:
             print(f"  {metric}: {value} bytes")
        else:
             print(f"  {metric}: {value}%")

#### `payment_installment`

In [ ]:
# Get the list of order IDs
order_ids = orderp['payment_installment'].tolist()

# Function to benchmark a single search/sort combination
def benchmark_combination(search_func, sort_func, data, data_to_search):
    # Measure sorting time and memory
    tracemalloc.start()
    start_time_sort = time.time()
    sorted_data = sort_func(data.copy()) # Sort a copy to not modify original data
    end_time_sort = time.time()
    memory_used_sort = tracemalloc.get_traced_memory()[1]
    tracemalloc.stop()

    # Measure searching time and memory for each item to search
    search_times = []
    search_memory_usages = []
    for item in data_to_search:
        tracemalloc.start()
        start_time_search = time.time()
        # Search function needs sorted data if it's a comparison-based search
        if search_func in [jump_search, binary_search]:
             result = search_func(sorted_data, item)
        else: # Hash search doesn't require sorted data, but we use the same sorted copy for consistency in benchmark
             result = search_func(sorted_data, item)
        end_time_search = time.time()
        memory_used_search = tracemalloc.get_traced_memory()[1]
        tracemalloc.stop()
        search_times.append(end_time_search - start_time_search)
        search_memory_usages.append(memory_used_search)

    # Average search time and memory
    avg_search_time = sum(search_times) / len(search_times) if search_times else 0
    avg_search_memory = sum(search_memory_usages) / len(search_memory_usages) if search_memory_usages else 0

    # Measure CPU usage (approximate by checking process CPU percentage after operations)
    # This is a rough measure and might not be precise for very short operations.
    # A more precise method would involve sampling CPU usage during the operation.
    process = psutil.Process()
    cpu_percent_after = process.cpu_percent(interval=None) # Non-blocking

    return {
        "Sort Time": end_time_sort - start_time_sort,
        "Sort Memory (bytes)": memory_used_sort,
        "Avg Search Time": avg_search_time,
        "Avg Search Memory (bytes)": avg_search_memory,
        "Approx CPU Usage (%)": cpu_percent_after
    }

# Data to search (a random sample of order_ids)
num_items_to_search = min(1000, len(order_ids)) # Search for up to 1000 random items
items_to_search = random.sample(order_ids, num_items_to_search) if num_items_to_search > 0 else []

# Benchmark each combination
benchmarking_results = {}

for name, (search_func, sort_func) in combinations.items():
    print(f"Benchmarking {name}...")
    # Pass a copy of the data to each benchmark to ensure a clean state
    results = benchmark_combination(search_func, sort_func, order_ids, items_to_search)
    benchmarking_results[name] = results
    print(f"  Sort Time: {results['Sort Time']:.6f} seconds")
    print(f"  Sort Memory: {results['Sort Memory (bytes)']} bytes")
    print(f"  Avg Search Time: {results['Avg Search Time']:.9f} seconds")
    print(f"  Avg Search Memory: {results['Avg Search Memory (bytes)']} bytes")
    print(f"  Approx CPU Usage: {results['Approx CPU Usage (%)']}%")
    print("-" * 30)

# Display the results in a structured format
print("\n--- Benchmarking Results ---")
for name, results in benchmarking_results.items():
    print(f"\nCombination: {name}")
    for metric, value in results.items():
        if "Time" in metric:
            print(f"  {metric}: {value:.9f} seconds")
        elif "Memory" in metric:
             print(f"  {metric}: {value} bytes")
        else:
             print(f"  {metric}: {value}%")

#### `payment_value`

In [ ]:
# Get the list of order IDs
order_ids = orderp['payment_value'].tolist()

# Function to benchmark a single search/sort combination
def benchmark_combination(search_func, sort_func, data, data_to_search):
    # Measure sorting time and memory
    tracemalloc.start()
    start_time_sort = time.time()
    sorted_data = sort_func(data.copy()) # Sort a copy to not modify original data
    end_time_sort = time.time()
    memory_used_sort = tracemalloc.get_traced_memory()[1]
    tracemalloc.stop()

    # Measure searching time and memory for each item to search
    search_times = []
    search_memory_usages = []
    for item in data_to_search:
        tracemalloc.start()
        start_time_search = time.time()
        # Search function needs sorted data if it's a comparison-based search
        if search_func in [jump_search, binary_search]:
             result = search_func(sorted_data, item)
        else: # Hash search doesn't require sorted data, but we use the same sorted copy for consistency in benchmark
             result = search_func(sorted_data, item)
        end_time_search = time.time()
        memory_used_search = tracemalloc.get_traced_memory()[1]
        tracemalloc.stop()
        search_times.append(end_time_search - start_time_search)
        search_memory_usages.append(memory_used_search)

    # Average search time and memory
    avg_search_time = sum(search_times) / len(search_times) if search_times else 0
    avg_search_memory = sum(search_memory_usages) / len(search_memory_usages) if search_memory_usages else 0

    # Measure CPU usage (approximate by checking process CPU percentage after operations)
    # This is a rough measure and might not be precise for very short operations.
    # A more precise method would involve sampling CPU usage during the operation.
    process = psutil.Process()
    cpu_percent_after = process.cpu_percent(interval=None) # Non-blocking

    return {
        "Sort Time": end_time_sort - start_time_sort,
        "Sort Memory (bytes)": memory_used_sort,
        "Avg Search Time": avg_search_time,
        "Avg Search Memory (bytes)": avg_search_memory,
        "Approx CPU Usage (%)": cpu_percent_after
    }

# Data to search (a random sample of order_ids)
num_items_to_search = min(1000, len(order_ids)) # Search for up to 1000 random items
items_to_search = random.sample(order_ids, num_items_to_search) if num_items_to_search > 0 else []

# Benchmark each combination
benchmarking_results = {}

for name, (search_func, sort_func) in combinations.items():
    print(f"Benchmarking {name}...")
    # Pass a copy of the data to each benchmark to ensure a clean state
    results = benchmark_combination(search_func, sort_func, order_ids, items_to_search)
    benchmarking_results[name] = results
    print(f"  Sort Time: {results['Sort Time']:.6f} seconds")
    print(f"  Sort Memory: {results['Sort Memory (bytes)']} bytes")
    print(f"  Avg Search Time: {results['Avg Search Time']:.9f} seconds")
    print(f"  Avg Search Memory: {results['Avg Search Memory (bytes)']} bytes")
    print(f"  Approx CPU Usage: {results['Approx CPU Usage (%)']}%")
    print("-" * 30)

# Display the results in a structured format
print("\n--- Benchmarking Results ---")
for name, results in benchmarking_results.items():
    print(f"\nCombination: {name}")
    for metric, value in results.items():
        if "Time" in metric:
            print(f"  {metric}: {value:.9f} seconds")
        elif "Memory" in metric:
             print(f"  {metric}: {value} bytes")
        else:
             print(f"  {metric}: {value}%")

### Order Review Dataset Benchmarking

In [ ]:
orderr.info()

#### `review_id`

In [ ]:
# Get the list of order IDs
order_ids = orderr['review_id'].tolist()

# Function to benchmark a single search/sort combination
def benchmark_combination(search_func, sort_func, data, data_to_search):
    # Measure sorting time and memory
    tracemalloc.start()
    start_time_sort = time.time()
    sorted_data = sort_func(data.copy()) # Sort a copy to not modify original data
    end_time_sort = time.time()
    memory_used_sort = tracemalloc.get_traced_memory()[1]
    tracemalloc.stop()

    # Measure searching time and memory for each item to search
    search_times = []
    search_memory_usages = []
    for item in data_to_search:
        tracemalloc.start()
        start_time_search = time.time()
        # Search function needs sorted data if it's a comparison-based search
        if search_func in [jump_search, binary_search]:
             result = search_func(sorted_data, item)
        else: # Hash search doesn't require sorted data, but we use the same sorted copy for consistency in benchmark
             result = search_func(sorted_data, item)
        end_time_search = time.time()
        memory_used_search = tracemalloc.get_traced_memory()[1]
        tracemalloc.stop()
        search_times.append(end_time_search - start_time_search)
        search_memory_usages.append(memory_used_search)

    # Average search time and memory
    avg_search_time = sum(search_times) / len(search_times) if search_times else 0
    avg_search_memory = sum(search_memory_usages) / len(search_memory_usages) if search_memory_usages else 0

    # Measure CPU usage (approximate by checking process CPU percentage after operations)
    # This is a rough measure and might not be precise for very short operations.
    # A more precise method would involve sampling CPU usage during the operation.
    process = psutil.Process()
    cpu_percent_after = process.cpu_percent(interval=None) # Non-blocking

    return {
        "Sort Time": end_time_sort - start_time_sort,
        "Sort Memory (bytes)": memory_used_sort,
        "Avg Search Time": avg_search_time,
        "Avg Search Memory (bytes)": avg_search_memory,
        "Approx CPU Usage (%)": cpu_percent_after
    }

# Data to search (a random sample of order_ids)
num_items_to_search = min(1000, len(order_ids)) # Search for up to 1000 random items
items_to_search = random.sample(order_ids, num_items_to_search) if num_items_to_search > 0 else []

# Benchmark each combination
benchmarking_results = {}

for name, (search_func, sort_func) in combinations.items():
    print(f"Benchmarking {name}...")
    # Pass a copy of the data to each benchmark to ensure a clean state
    results = benchmark_combination(search_func, sort_func, order_ids, items_to_search)
    benchmarking_results[name] = results
    print(f"  Sort Time: {results['Sort Time']:.6f} seconds")
    print(f"  Sort Memory: {results['Sort Memory (bytes)']} bytes")
    print(f"  Avg Search Time: {results['Avg Search Time']:.9f} seconds")
    print(f"  Avg Search Memory: {results['Avg Search Memory (bytes)']} bytes")
    print(f"  Approx CPU Usage: {results['Approx CPU Usage (%)']}%")
    print("-" * 30)

# Display the results in a structured format
print("\n--- Benchmarking Results ---")
for name, results in benchmarking_results.items():
    print(f"\nCombination: {name}")
    for metric, value in results.items():
        if "Time" in metric:
            print(f"  {metric}: {value:.9f} seconds")
        elif "Memory" in metric:
             print(f"  {metric}: {value} bytes")
        else:
             print(f"  {metric}: {value}%")

#### `order_id`

In [ ]:
# Get the list of order IDs
order_ids = orderr['order_id'].tolist()

# Function to benchmark a single search/sort combination
def benchmark_combination(search_func, sort_func, data, data_to_search):
    # Measure sorting time and memory
    tracemalloc.start()
    start_time_sort = time.time()
    sorted_data = sort_func(data.copy()) # Sort a copy to not modify original data
    end_time_sort = time.time()
    memory_used_sort = tracemalloc.get_traced_memory()[1]
    tracemalloc.stop()

    # Measure searching time and memory for each item to search
    search_times = []
    search_memory_usages = []
    for item in data_to_search:
        tracemalloc.start()
        start_time_search = time.time()
        # Search function needs sorted data if it's a comparison-based search
        if search_func in [jump_search, binary_search]:
             result = search_func(sorted_data, item)
        else: # Hash search doesn't require sorted data, but we use the same sorted copy for consistency in benchmark
             result = search_func(sorted_data, item)
        end_time_search = time.time()
        memory_used_search = tracemalloc.get_traced_memory()[1]
        tracemalloc.stop()
        search_times.append(end_time_search - start_time_search)
        search_memory_usages.append(memory_used_search)

    # Average search time and memory
    avg_search_time = sum(search_times) / len(search_times) if search_times else 0
    avg_search_memory = sum(search_memory_usages) / len(search_memory_usages) if search_memory_usages else 0

    # Measure CPU usage (approximate by checking process CPU percentage after operations)
    # This is a rough measure and might not be precise for very short operations.
    # A more precise method would involve sampling CPU usage during the operation.
    process = psutil.Process()
    cpu_percent_after = process.cpu_percent(interval=None) # Non-blocking

    return {
        "Sort Time": end_time_sort - start_time_sort,
        "Sort Memory (bytes)": memory_used_sort,
        "Avg Search Time": avg_search_time,
        "Avg Search Memory (bytes)": avg_search_memory,
        "Approx CPU Usage (%)": cpu_percent_after
    }

# Data to search (a random sample of order_ids)
num_items_to_search = min(1000, len(order_ids)) # Search for up to 1000 random items
items_to_search = random.sample(order_ids, num_items_to_search) if num_items_to_search > 0 else []

# Benchmark each combination
benchmarking_results = {}

for name, (search_func, sort_func) in combinations.items():
    print(f"Benchmarking {name}...")
    # Pass a copy of the data to each benchmark to ensure a clean state
    results = benchmark_combination(search_func, sort_func, order_ids, items_to_search)
    benchmarking_results[name] = results
    print(f"  Sort Time: {results['Sort Time']:.6f} seconds")
    print(f"  Sort Memory: {results['Sort Memory (bytes)']} bytes")
    print(f"  Avg Search Time: {results['Avg Search Time']:.9f} seconds")
    print(f"  Avg Search Memory: {results['Avg Search Memory (bytes)']} bytes")
    print(f"  Approx CPU Usage: {results['Approx CPU Usage (%)']}%")
    print("-" * 30)

# Display the results in a structured format
print("\n--- Benchmarking Results ---")
for name, results in benchmarking_results.items():
    print(f"\nCombination: {name}")
    for metric, value in results.items():
        if "Time" in metric:
            print(f"  {metric}: {value:.9f} seconds")
        elif "Memory" in metric:
             print(f"  {metric}: {value} bytes")
        else:
             print(f"  {metric}: {value}%")

#### `review_score`

In [ ]:
# Get the list of order IDs
order_ids = orderr['review_score'].tolist()

# Function to benchmark a single search/sort combination
def benchmark_combination(search_func, sort_func, data, data_to_search):
    # Measure sorting time and memory
    tracemalloc.start()
    start_time_sort = time.time()
    sorted_data = sort_func(data.copy()) # Sort a copy to not modify original data
    end_time_sort = time.time()
    memory_used_sort = tracemalloc.get_traced_memory()[1]
    tracemalloc.stop()

    # Measure searching time and memory for each item to search
    search_times = []
    search_memory_usages = []
    for item in data_to_search:
        tracemalloc.start()
        start_time_search = time.time()
        # Search function needs sorted data if it's a comparison-based search
        if search_func in [jump_search, binary_search]:
             result = search_func(sorted_data, item)
        else: # Hash search doesn't require sorted data, but we use the same sorted copy for consistency in benchmark
             result = search_func(sorted_data, item)
        end_time_search = time.time()
        memory_used_search = tracemalloc.get_traced_memory()[1]
        tracemalloc.stop()
        search_times.append(end_time_search - start_time_search)
        search_memory_usages.append(memory_used_search)

    # Average search time and memory
    avg_search_time = sum(search_times) / len(search_times) if search_times else 0
    avg_search_memory = sum(search_memory_usages) / len(search_memory_usages) if search_memory_usages else 0

    # Measure CPU usage (approximate by checking process CPU percentage after operations)
    # This is a rough measure and might not be precise for very short operations.
    # A more precise method would involve sampling CPU usage during the operation.
    process = psutil.Process()
    cpu_percent_after = process.cpu_percent(interval=None) # Non-blocking

    return {
        "Sort Time": end_time_sort - start_time_sort,
        "Sort Memory (bytes)": memory_used_sort,
        "Avg Search Time": avg_search_time,
        "Avg Search Memory (bytes)": avg_search_memory,
        "Approx CPU Usage (%)": cpu_percent_after
    }

# Data to search (a random sample of order_ids)
num_items_to_search = min(1000, len(order_ids)) # Search for up to 1000 random items
items_to_search = random.sample(order_ids, num_items_to_search) if num_items_to_search > 0 else []

# Benchmark each combination
benchmarking_results = {}

for name, (search_func, sort_func) in combinations.items():
    print(f"Benchmarking {name}...")
    # Pass a copy of the data to each benchmark to ensure a clean state
    results = benchmark_combination(search_func, sort_func, order_ids, items_to_search)
    benchmarking_results[name] = results
    print(f"  Sort Time: {results['Sort Time']:.6f} seconds")
    print(f"  Sort Memory: {results['Sort Memory (bytes)']} bytes")
    print(f"  Avg Search Time: {results['Avg Search Time']:.9f} seconds")
    print(f"  Avg Search Memory: {results['Avg Search Memory (bytes)']} bytes")
    print(f"  Approx CPU Usage: {results['Approx CPU Usage (%)']}%")
    print("-" * 30)

# Display the results in a structured format
print("\n--- Benchmarking Results ---")
for name, results in benchmarking_results.items():
    print(f"\nCombination: {name}")
    for metric, value in results.items():
        if "Time" in metric:
            print(f"  {metric}: {value:.9f} seconds")
        elif "Memory" in metric:
             print(f"  {metric}: {value} bytes")
        else:
             print(f"  {metric}: {value}%")

#### `review_comment_title`

In [ ]:
# Get the list of order IDs
order_ids = orderr['review_comment_title'].tolist()

# Function to benchmark a single search/sort combination
def benchmark_combination(search_func, sort_func, data, data_to_search):
    # Measure sorting time and memory
    tracemalloc.start()
    start_time_sort = time.time()
    sorted_data = sort_func(data.copy()) # Sort a copy to not modify original data
    end_time_sort = time.time()
    memory_used_sort = tracemalloc.get_traced_memory()[1]
    tracemalloc.stop()

    # Measure searching time and memory for each item to search
    search_times = []
    search_memory_usages = []
    for item in data_to_search:
        tracemalloc.start()
        start_time_search = time.time()
        # Search function needs sorted data if it's a comparison-based search
        if search_func in [jump_search, binary_search]:
             result = search_func(sorted_data, item)
        else: # Hash search doesn't require sorted data, but we use the same sorted copy for consistency in benchmark
             result = search_func(sorted_data, item)
        end_time_search = time.time()
        memory_used_search = tracemalloc.get_traced_memory()[1]
        tracemalloc.stop()
        search_times.append(end_time_search - start_time_search)
        search_memory_usages.append(memory_used_search)

    # Average search time and memory
    avg_search_time = sum(search_times) / len(search_times) if search_times else 0
    avg_search_memory = sum(search_memory_usages) / len(search_memory_usages) if search_memory_usages else 0

    # Measure CPU usage (approximate by checking process CPU percentage after operations)
    # This is a rough measure and might not be precise for very short operations.
    # A more precise method would involve sampling CPU usage during the operation.
    process = psutil.Process()
    cpu_percent_after = process.cpu_percent(interval=None) # Non-blocking

    return {
        "Sort Time": end_time_sort - start_time_sort,
        "Sort Memory (bytes)": memory_used_sort,
        "Avg Search Time": avg_search_time,
        "Avg Search Memory (bytes)": avg_search_memory,
        "Approx CPU Usage (%)": cpu_percent_after
    }

# Data to search (a random sample of order_ids)
num_items_to_search = min(1000, len(order_ids)) # Search for up to 1000 random items
items_to_search = random.sample(order_ids, num_items_to_search) if num_items_to_search > 0 else []

# Benchmark each combination
benchmarking_results = {}

for name, (search_func, sort_func) in combinations.items():
    print(f"Benchmarking {name}...")
    # Pass a copy of the data to each benchmark to ensure a clean state
    results = benchmark_combination(search_func, sort_func, order_ids, items_to_search)
    benchmarking_results[name] = results
    print(f"  Sort Time: {results['Sort Time']:.6f} seconds")
    print(f"  Sort Memory: {results['Sort Memory (bytes)']} bytes")
    print(f"  Avg Search Time: {results['Avg Search Time']:.9f} seconds")
    print(f"  Avg Search Memory: {results['Avg Search Memory (bytes)']} bytes")
    print(f"  Approx CPU Usage: {results['Approx CPU Usage (%)']}%")
    print("-" * 30)

# Display the results in a structured format
print("\n--- Benchmarking Results ---")
for name, results in benchmarking_results.items():
    print(f"\nCombination: {name}")
    for metric, value in results.items():
        if "Time" in metric:
            print(f"  {metric}: {value:.9f} seconds")
        elif "Memory" in metric:
             print(f"  {metric}: {value} bytes")
        else:
             print(f"  {metric}: {value}%")

#### `review_comment_message`

In [ ]:
# Get the list of order IDs
order_ids = orderr['review_comment_message'].tolist()

# Function to benchmark a single search/sort combination
def benchmark_combination(search_func, sort_func, data, data_to_search):
    # Measure sorting time and memory
    tracemalloc.start()
    start_time_sort = time.time()
    sorted_data = sort_func(data.copy()) # Sort a copy to not modify original data
    end_time_sort = time.time()
    memory_used_sort = tracemalloc.get_traced_memory()[1]
    tracemalloc.stop()

    # Measure searching time and memory for each item to search
    search_times = []
    search_memory_usages = []
    for item in data_to_search:
        tracemalloc.start()
        start_time_search = time.time()
        # Search function needs sorted data if it's a comparison-based search
        if search_func in [jump_search, binary_search]:
             result = search_func(sorted_data, item)
        else: # Hash search doesn't require sorted data, but we use the same sorted copy for consistency in benchmark
             result = search_func(sorted_data, item)
        end_time_search = time.time()
        memory_used_search = tracemalloc.get_traced_memory()[1]
        tracemalloc.stop()
        search_times.append(end_time_search - start_time_search)
        search_memory_usages.append(memory_used_search)

    # Average search time and memory
    avg_search_time = sum(search_times) / len(search_times) if search_times else 0
    avg_search_memory = sum(search_memory_usages) / len(search_memory_usages) if search_memory_usages else 0

    # Measure CPU usage (approximate by checking process CPU percentage after operations)
    # This is a rough measure and might not be precise for very short operations.
    # A more precise method would involve sampling CPU usage during the operation.
    process = psutil.Process()
    cpu_percent_after = process.cpu_percent(interval=None) # Non-blocking

    return {
        "Sort Time": end_time_sort - start_time_sort,
        "Sort Memory (bytes)": memory_used_sort,
        "Avg Search Time": avg_search_time,
        "Avg Search Memory (bytes)": avg_search_memory,
        "Approx CPU Usage (%)": cpu_percent_after
    }

# Data to search (a random sample of order_ids)
num_items_to_search = min(1000, len(order_ids)) # Search for up to 1000 random items
items_to_search = random.sample(order_ids, num_items_to_search) if num_items_to_search > 0 else []

# Benchmark each combination
benchmarking_results = {}

for name, (search_func, sort_func) in combinations.items():
    print(f"Benchmarking {name}...")
    # Pass a copy of the data to each benchmark to ensure a clean state
    results = benchmark_combination(search_func, sort_func, order_ids, items_to_search)
    benchmarking_results[name] = results
    print(f"  Sort Time: {results['Sort Time']:.6f} seconds")
    print(f"  Sort Memory: {results['Sort Memory (bytes)']} bytes")
    print(f"  Avg Search Time: {results['Avg Search Time']:.9f} seconds")
    print(f"  Avg Search Memory: {results['Avg Search Memory (bytes)']} bytes")
    print(f"  Approx CPU Usage: {results['Approx CPU Usage (%)']}%")
    print("-" * 30)

# Display the results in a structured format
print("\n--- Benchmarking Results ---")
for name, results in benchmarking_results.items():
    print(f"\nCombination: {name}")
    for metric, value in results.items():
        if "Time" in metric:
            print(f"  {metric}: {value:.9f} seconds")
        elif "Memory" in metric:
             print(f"  {metric}: {value} bytes")
        else:
             print(f"  {metric}: {value}%")

 #### `review_creation_date`

In [ ]:
# Get the list of order IDs
order_ids = orderp['review_creation_date'].tolist()

# Function to benchmark a single search/sort combination
def benchmark_combination(search_func, sort_func, data, data_to_search):
    # Measure sorting time and memory
    tracemalloc.start()
    start_time_sort = time.time()
    sorted_data = sort_func(data.copy()) # Sort a copy to not modify original data
    end_time_sort = time.time()
    memory_used_sort = tracemalloc.get_traced_memory()[1]
    tracemalloc.stop()

    # Measure searching time and memory for each item to search
    search_times = []
    search_memory_usages = []
    for item in data_to_search:
        tracemalloc.start()
        start_time_search = time.time()
        # Search function needs sorted data if it's a comparison-based search
        if search_func in [jump_search, binary_search]:
             result = search_func(sorted_data, item)
        else: # Hash search doesn't require sorted data, but we use the same sorted copy for consistency in benchmark
             result = search_func(sorted_data, item)
        end_time_search = time.time()
        memory_used_search = tracemalloc.get_traced_memory()[1]
        tracemalloc.stop()
        search_times.append(end_time_search - start_time_search)
        search_memory_usages.append(memory_used_search)

    # Average search time and memory
    avg_search_time = sum(search_times) / len(search_times) if search_times else 0
    avg_search_memory = sum(search_memory_usages) / len(search_memory_usages) if search_memory_usages else 0

    # Measure CPU usage (approximate by checking process CPU percentage after operations)
    # This is a rough measure and might not be precise for very short operations.
    # A more precise method would involve sampling CPU usage during the operation.
    process = psutil.Process()
    cpu_percent_after = process.cpu_percent(interval=None) # Non-blocking

    return {
        "Sort Time": end_time_sort - start_time_sort,
        "Sort Memory (bytes)": memory_used_sort,
        "Avg Search Time": avg_search_time,
        "Avg Search Memory (bytes)": avg_search_memory,
        "Approx CPU Usage (%)": cpu_percent_after
    }

# Data to search (a random sample of order_ids)
num_items_to_search = min(1000, len(order_ids)) # Search for up to 1000 random items
items_to_search = random.sample(order_ids, num_items_to_search) if num_items_to_search > 0 else []

# Benchmark each combination
benchmarking_results = {}

for name, (search_func, sort_func) in combinations.items():
    print(f"Benchmarking {name}...")
    # Pass a copy of the data to each benchmark to ensure a clean state
    results = benchmark_combination(search_func, sort_func, order_ids, items_to_search)
    benchmarking_results[name] = results
    print(f"  Sort Time: {results['Sort Time']:.6f} seconds")
    print(f"  Sort Memory: {results['Sort Memory (bytes)']} bytes")
    print(f"  Avg Search Time: {results['Avg Search Time']:.9f} seconds")
    print(f"  Avg Search Memory: {results['Avg Search Memory (bytes)']} bytes")
    print(f"  Approx CPU Usage: {results['Approx CPU Usage (%)']}%")
    print("-" * 30)

# Display the results in a structured format
print("\n--- Benchmarking Results ---")
for name, results in benchmarking_results.items():
    print(f"\nCombination: {name}")
    for metric, value in results.items():
        if "Time" in metric:
            print(f"  {metric}: {value:.9f} seconds")
        elif "Memory" in metric:
             print(f"  {metric}: {value} bytes")
        else:
             print(f"  {metric}: {value}%")

 #### `review_answer_timestamp`

In [ ]:
# Get the list of order IDs
order_ids = orderp['review_answer_timestamp'].tolist()

# Function to benchmark a single search/sort combination
def benchmark_combination(search_func, sort_func, data, data_to_search):
    # Measure sorting time and memory
    tracemalloc.start()
    start_time_sort = time.time()
    sorted_data = sort_func(data.copy()) # Sort a copy to not modify original data
    end_time_sort = time.time()
    memory_used_sort = tracemalloc.get_traced_memory()[1]
    tracemalloc.stop()

    # Measure searching time and memory for each item to search
    search_times = []
    search_memory_usages = []
    for item in data_to_search:
        tracemalloc.start()
        start_time_search = time.time()
        # Search function needs sorted data if it's a comparison-based search
        if search_func in [jump_search, binary_search]:
             result = search_func(sorted_data, item)
        else: # Hash search doesn't require sorted data, but we use the same sorted copy for consistency in benchmark
             result = search_func(sorted_data, item)
        end_time_search = time.time()
        memory_used_search = tracemalloc.get_traced_memory()[1]
        tracemalloc.stop()
        search_times.append(end_time_search - start_time_search)
        search_memory_usages.append(memory_used_search)

    # Average search time and memory
    avg_search_time = sum(search_times) / len(search_times) if search_times else 0
    avg_search_memory = sum(search_memory_usages) / len(search_memory_usages) if search_memory_usages else 0

    # Measure CPU usage (approximate by checking process CPU percentage after operations)
    # This is a rough measure and might not be precise for very short operations.
    # A more precise method would involve sampling CPU usage during the operation.
    process = psutil.Process()
    cpu_percent_after = process.cpu_percent(interval=None) # Non-blocking

    return {
        "Sort Time": end_time_sort - start_time_sort,
        "Sort Memory (bytes)": memory_used_sort,
        "Avg Search Time": avg_search_time,
        "Avg Search Memory (bytes)": avg_search_memory,
        "Approx CPU Usage (%)": cpu_percent_after
    }

# Data to search (a random sample of order_ids)
num_items_to_search = min(1000, len(order_ids)) # Search for up to 1000 random items
items_to_search = random.sample(order_ids, num_items_to_search) if num_items_to_search > 0 else []

# Benchmark each combination
benchmarking_results = {}

for name, (search_func, sort_func) in combinations.items():
    print(f"Benchmarking {name}...")
    # Pass a copy of the data to each benchmark to ensure a clean state
    results = benchmark_combination(search_func, sort_func, order_ids, items_to_search)
    benchmarking_results[name] = results
    print(f"  Sort Time: {results['Sort Time']:.6f} seconds")
    print(f"  Sort Memory: {results['Sort Memory (bytes)']} bytes")
    print(f"  Avg Search Time: {results['Avg Search Time']:.9f} seconds")
    print(f"  Avg Search Memory: {results['Avg Search Memory (bytes)']} bytes")
    print(f"  Approx CPU Usage: {results['Approx CPU Usage (%)']}%")
    print("-" * 30)

# Display the results in a structured format
print("\n--- Benchmarking Results ---")
for name, results in benchmarking_results.items():
    print(f"\nCombination: {name}")
    for metric, value in results.items():
        if "Time" in metric:
            print(f"  {metric}: {value:.9f} seconds")
        elif "Memory" in metric:
             print(f"  {metric}: {value} bytes")
        else:
             print(f"  {metric}: {value}%")